In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # langsmith tracking
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")

In [6]:
!ollama run gemma:2b

⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ >>> Send a message (/? for help)
Use Ctrl + d or /bye to exit.
>>> Send a message (/? for help)
>>> Send a message (/? for help)

In [9]:
# Data Ingestion (scrape data from webpage)

from langchain_community.document_loaders import WebBaseLoader


In [10]:
loader = WebBaseLoader(
    "https://docs.smith.langchain.com/administration/concepts"
)

In [11]:
loader.load()

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content="\n\n\n\n\nConcepts | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationConceptual GuideOn this pageConcepts\nThis conceptual guide covers topics related to managing users, organizations, and workspaces within L

In [12]:
## Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = splitter.split_documents(loader.load())

In [13]:
## Embedding creation
from langchain_community.embeddings import OllamaEmbeddings
embeddings = (
    OllamaEmbeddings(
        model = "gemma:2b"
    )
)

/var/folders/jj/z9jhw1z95m5bwpgdbhgbvct40000gn/T/ipykernel_36734/3058066653.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(


In [14]:
## Vector store creation

from langchain_community.vectorstores import FAISS 

db = FAISS.from_documents(
    documents,
    embeddings
)

In [15]:
## query from vector store

query = "Roles are used to define the set of permissions that a user has within a workspace."

result = db.similarity_search(
    query=query
)

In [16]:
result[0].page_content

'Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces. An Organization Admin has Admin access to all workspaces in an organization\nOrganization User may read organization information but cannot execute any write actions at the organization level. An Organization User can be added to a subset of workspaces and assigned workspace roles as usual (if RBAC is enabled), which specify permissions at the workspace level.'

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# prompt creation
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", """Answer the following questions based only on the provided context:
<context>
{context}
</context>""")
])

# llm setup
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="gemma:2b")

# retrival chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

from langchain_core.documents import Document
result = document_chain.invoke(
    {
        "input":query,
        "context": [
            Document(page_content=query)
        ]
    }
)

In [18]:
result

'Sure, I understand the context and can answer your questions based on the provided context. \n\nPlease ask your questions.'

In [19]:
## Retriver
retriver = db.as_retriever()

from langchain.chains import create_retrieval_chain 
retriever_chain = create_retrieval_chain(
    retriver,
    document_chain
)

In [20]:
response = retriever_chain.invoke(
    {
        "input":query
    }
)

In [21]:
response

{'input': 'Roles are used to define the set of permissions that a user has within a workspace.',
 'context': [Document(id='104c3322-c27e-4338-942e-4d2632f74aab', metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content='Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces. An Organization Admin has Admin access to all workspaces in an organization\nOrganization User may read organization information but cannot execute any write actions at the organization level. An Organization User can be added to a subset of workspaces and assigned workspace roles as usual (if RBAC is enabled), which specify permissions at the workspace level.'),
  Document(id='a564fb50-8f28-4cb8-ae82-d73d565ff0c7', metadata={'source': 'https

In [22]:
response["answer"]

'Sure, here are the answers to the questions:\n\n1. **What is the difference between a personal organization and a shared organization?**\n - A personal organization is created for you automatically when you sign up.\n - A shared organization is created by you and can be shared with others.\n\n\n2. **What is the purpose of workspace roles?**\n - Workspace roles are used to specify permissions at the workspace level, which define what users can do and access in that specific workspace.\n\n\n3. **What are the organization roles?**\n - The Organization User role is only available in organizations with multiple workspaces. In organizations limited to a single workspace, all users are Organization Admins.\n\n\n4. **What is the purpose of workspace roles?**\n - Workspace roles allow you to define permissions at the workspace level, which define what users can do and access in that specific workspace.\n\n\n5. **What is the role of the Organization Admin in an organization?**\n - An Organizati